In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import http

drive.mount('/content/drive')

# 시가총액 상위 300개 기업리스트 (미래에셋 대우 API 미확인으로, 2021.09.12 기준 시총 상위 300개 기업 사용)
lists = pd.read_csv("/content/drive/MyDrive/미래에셋/kospi/KOSPI_시가총액_상위500.csv")
lists_1 = lists.loc[0:299]
lists_2 = lists_1[['종목명']]


# ESG 키워드
ESG = {
'E_keyword' : ['친환경', '탄소', '에너지'],
'S_keyword' : ['캠페인', '교육', '사회'],
'G_keyword' : ['위원회', '이사', '지배구조']}
ESG_keyword = pd.DataFrame.from_dict(ESG)

print(ESG_keyword)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  E_keyword S_keyword G_keyword
0       친환경       캠페인       위원회
1        탄소        교육        이사
2       에너지        사회      지배구조


In [ ]:
# 지난 N개월 간의 종목명 + E 관련 뉴스기사 본문 추출
from multiprocessing import Pool
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
import urllib.request
from urllib.parse import quote
import pandas as pd


# 크롤링 메인 함수
def get_news(stock, keyword, query, sort, lastpage, period):

  stockName = stock
  ESGKeyword = keyword

  # 크롤링 결과 저장 데이터프레임
  news_df = pd.DataFrame(columns=("stockName", "ESGKeyword", "Title", "Article")) 

  idx = 0


  for month in range (0, period): # 해당 검색어(query)로 N(=period)개월치 기사 추출

    s_date = datetime.date.today() + timedelta(days = -30 * (month+1))
    e_date = datetime.date.today() + timedelta(days = -30 * month)

    pageNum = 1

    for i in range (1, int(lastpage)*10, 10):
      # print(f"/{pageNum}페이지입니다/")

      url_query = quote(query)
      url = f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={url_query}&sort=0&photo=0&field=0&pd=6&ds={s_date}&de={e_date}&nso=so:r,p:{period}m,a:all&start={i}"

      search_url = urllib.request.urlopen(url).read()
      soup = BeautifulSoup(search_url, 'html.parser')
      links = soup.find_all('div', {'class':'info_group'})
  

      # 하나의 검색 페이지 안에서의 반복
      for link in links:

        # '네이버 뉴스' 형식으로 작성된 기사만을 추출 (href 2번)
        news_urls = link.find_all('a')

        if (len(news_urls) == 1): # '네이버 뉴스' 형식 X
          continue

        else:  # '네이버 뉴스' 형식 O
          
          for news_url_test in news_urls:
            news_url = news_url_test.get('href')

          try:
            news_link = urllib.request.urlopen(news_url).read()
          except (http.client.IncompleteRead) as e:
            news_link = e.partial
          
          news_html = BeautifulSoup(news_link, 'html.parser')

          try: 
            title = news_html.find('h3', {'id': 'articleTitle'}).get_text()
            article = news_html.find('div', {'id': 'articleBodyContents'}).get_text()
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
            article = article.replace("function _flash_removeCallback() {}", "")
            article = article.replace("\n","")
            article = article.replace("\t","")

            news_df.loc[idx] = [stockName, ESGKeyword, title, article]
            idx += 1

          except: 
            pass

          # print("#", end="")


      pageNum += 1


  return news_df




In [ ]:
# E(환경)부문


# CONCAT 목적의 초기 데이터프레임 형성
default = {
'stockName' : ['Test'],
'ESGKeyword' : ['Test'],
'Title' : ['Test'],
'Article' : ['Test']}
df_default = pd.DataFrame.from_dict(default) 


# 파라미터 입력 후 크롤링
for i in range(50,100): #시총 상위 50개 기업

  query = lists_2['종목명'].values[i] + "+" + ESG_keyword['E_keyword'].values[2]
  sort = "0" # 관련도 순 기사
  lastpage = 100 # 100 페이지까지 크롤링 (최대 400페이지까지 가능하기는 하나, 파일 크기 과부하 방지)
  period = 6 # 6개월간의 기사

  print(lists_2['종목명'].values[i])
  print(ESG_keyword['E_keyword'].values[2])
  print(i)
  
  df = get_news(lists_2['종목명'].values[i], ESG_keyword['E_keyword'].values[2], query, sort, lastpage, period)
  df_default = pd.concat([df_default, df], axis = 0)


df_default.head()
df_default.to_csv('E부문_에너지_50_100.csv', header = True, index = False)

기업은행
에너지
50
LG디스플레이
에너지
51
현대제철
에너지
52
현대글로비스
에너지
53
SKC
에너지
54
CJ제일제당
에너지
55
LG유플러스
에너지
56
금호석유
에너지
57
강원랜드
에너지
58
현대건설
에너지
59
F&F
에너지
60
코웨이
에너지
61
미래에셋증권
에너지
62
한국타이어앤테크놀로지
에너지
63
LG이노텍
에너지
64
현대중공업지주
에너지
65
메리츠금융지주
에너지
66
맥쿼리인프라
에너지
67
한국금융지주
에너지
68
오리온
에너지
69
한미사이언스
에너지
70
이마트
에너지
71
유한양행
에너지
72
삼성엔지니어링
에너지
73
아모레G
에너지
74
삼성증권
에너지
75
쌍용C&E
에너지
76
DB손해보험
에너지
77
에스디바이오센서
에너지
78
한진칼
에너지
79
녹십자
에너지
80
두산밥캣
에너지
81
KCC
에너지
82
삼성중공업
에너지
83
메리츠화재
에너지
84
삼성카드
에너지
85
GS
에너지
86
CJ대한통운
에너지
87
팬오션
에너지
88
GS건설
에너지
89
NH투자증권
에너지
90
메리츠증권
에너지
91
롯데지주
에너지
92
한미약품
에너지
93
한솔케미칼
에너지
94
신풍제약
에너지
95
현대차2우B
에너지
96
GS리테일
에너지
97
일진머티리얼즈
에너지
98
SK케미칼
에너지
99


In [ ]:
df_default

,stockName,ESGKeyword,Title,Article
0,Test,Test,Test,Test
0,기업은행,에너지,"김두관, “산업·기업은행 본점, 전북혁신도시 이전 가능하다”","기사내용 요약새만금 재생에너지 산업 중심지, 균형발전 등 전북공약 발표법 개정 통해..."
1,기업은행,에너지,"IBK기업은행, 에너지업종 中企 최대 10억원 저금리 대출 지원",한국동서발전과 그린뉴딜분야 금융지원 동반성장 협약박주용 IBK기업은행 CIB그룹 부...
2,기업은행,에너지,"기업銀, 온실가스에너지 감축 목표 4년연속 초과달성",IBK기업은행이 온실가스에너지 목표관리제 감축 목표를 4년 연속 초과 달성했다. 기...
3,기업은행,에너지,"산업·기업은행, 녹색금융 선도한다면서 탈석탄은 '수수방관'",[탈석탄 금융]③탈석탄 선언 금융사 100곳 넘는데산업·기업은행만 여전히 '미적미적...
...,...,...,...,...
547,SK케미칼,에너지,"내가 버린 플라스틱 맞아?…“천연가스 추출하고 수소 만든다""",중국발 폐플라스틱 수입·사용 금지 여파로 폐플라스틱 처리에 골머리를 앓고 있다....
548,SK케미칼,에너지,"[2021 금융투자대상]삼성액티브자산운용, 친환경·디지털 트렌드 신속 반영",[이데일리 이은정 기자] 이데일리 2021 금융투자대상 국내주식 펀드 부문에 삼성액...
549,SK케미칼,에너지,中 플라스틱 제한 조치에… 국내 화학기업 생분해플라스틱 부각,중국 정부가 ‘일회용 플라스틱 제한·금지령’을 시행하면서 국내 화학 ...
550,SK케미칼,에너지,[2050 과학오디세이]전현우 서울시립대 자연과학연구소 연구원 “과학에도 철학적 성...,[주간경향] 과학이 있는 곳에는 언제나 ‘이만하면 충분하다’고 판단 내리는 시점이 ...


In [ ]:
df_default.to_csv('E부문_에너지_50_100.csv', header = True, index = False)